This example shows how to construct a CMD model from SFRs.
We will use PARESC stellar models and YBC bolometric corrections to sample the isochrones and assume a Kroupa (2001) IMF.

In [1]:
import StarFormationHistories as SFH
using StellarTracks: PARSECLibrary, isochrone
using BolometricCorrections: YBCGrid, filternames
using InitialMassFunctions: Kroupa2001
using TypedTables: Table, getproperties
using StaticArrays: SVector

The functions for generating mock CMDs for complex stellar populations take the mass in each SSP, not the SFR. To use SFRs, they need to be integrated over time bins to get the total mass per SSP.

In [2]:
ages = range(13.7e9, 0.1e9; step = -0.1e9) # Ages of SSPs in yr from 13.7 Gyr to 0.1 Gyr
sfrs = rand(length(ages)) .* 1e-3  # Random SFRs in solar masses / yr
# Need to define left bin edge for integrating SFRs;
# i.e., the first SFR bin goes from 13.76 -- 13.7 Gyr, next goes from 13.7 -- 13.6, and so on
T_max = 13.76e9
# Other properties for the stellar population
dmod = 25.0 # Distance modulus to add to sampled CMD
Av = 0.0 # V-band extinction to add to CMD; we will not add any

0.0

In [3]:
# Define the bins over which the SFR will be integrated, now in units of years
age_l = ages
age_h = vcat(T_max, age_l[begin:end-1])
[age_l age_h] # show bins

137×2 Matrix{Float64}:
 1.37e10  1.376e10
 1.36e10  1.37e10
 1.35e10  1.36e10
 1.34e10  1.35e10
 1.33e10  1.34e10
 1.32e10  1.33e10
 1.31e10  1.32e10
 1.3e10   1.31e10
 1.29e10  1.3e10
 1.28e10  1.29e10
 1.27e10  1.28e10
 1.26e10  1.27e10
 1.25e10  1.26e10
 ⋮        
 1.2e9    1.3e9
 1.1e9    1.2e9
 1.0e9    1.1e9
 9.0e8    1.0e9
 8.0e8    9.0e8
 7.0e8    8.0e8
 6.0e8    7.0e8
 5.0e8    6.0e8
 4.0e8    5.0e8
 3.0e8    4.0e8
 2.0e8    3.0e8
 1.0e8    2.0e8

With the time bins set up, we can now calculate the total stellar mass formed in each time bin by integrating, assuming fixed SFR within each bin.

In [4]:
ssp_masses = [sfrs[i] * (age_h[i] - age_l[i]) for i in eachindex(sfrs)]

137-element Vector{Float64}:
  1381.5838076642926
 36972.70383531356
 26100.59700926476
  6666.636151376615
 16285.151749411774
 76772.91597972874
 60898.52105765665
 10212.798348170849
 14775.128372943836
 46481.288036304424
 74162.4666728639
 37264.01047149853
  1892.4941245625828
     ⋮
 49882.37935925943
  2498.900727837294
 46853.4214062045
 77136.793841999
 43147.92301026891
 91260.70579409956
  3074.922392827961
 90027.97914405739
 66939.41161007513
  3745.463333390564
 34747.96588094662
 35485.27025308497

Now, assuming you have a vector for the metallicity at each SSP age, you can interpolate isochrones on-the-fly with StellarTracks.jl and BolometricCorrections.jl.

In [5]:
# Simple linear AMR model for simplicity; replace with whatever you want, same length as ages
MH = @. 0.1 * (13.76 .- (ages/1e9)) + -2.0

-1.994:0.01:-0.6339999999999999

In [6]:
# Load data for isochrone interpolation
tracklib = PARSECLibrary()
bcg = YBCGrid("acs_wfc"); # We'll use HST/ACS filters here

In [7]:
# Now interpolate appropriate isochrones. SSP ages have to be converted to logarithmic ages log10(age [yr]).
isos = [isochrone(tracklib, bcg, log10(ages[i]), MH[i], Av) for i in eachindex(ages)];

With the isochrones interpolated, we now have everything we need to construct the CMD. We will prepare inputs for `generate_stars_mass_composite`, then call it.

In [8]:
# Selects photometry columns from interpolated isochrone
function select_mags(iso, bcg)
    return select_mags(getproperties(iso, filternames(bcg)))
end
# Converts output from above into more efficient representation
function select_mags(subtable::Table{NamedTuple{B,NTuple{C,D}}}) where {B, C, D}
    return [SVector{C}(values(subtable[i])) for i in eachindex(subtable)]
end
# select_mags(isos[1], bcg)
mini_vec = [i.m_ini for i in isos] # Initial mass vectors from isochrones
mags = [select_mags(i, bcg) for i in isos] # Photometric magnitudes from isochrones
filters = collect(string.(filternames(bcg))) # Filter names for each photometric magnitude
mstar = sum(ssp_masses) # Total stellar mass
ssp_massfracs = ssp_masses ./ mstar # generate_stars_mass_composite requires fraction of mass in each SSP
imf = Kroupa2001(0.08, 100.0) # Kroupa 2001 IMF with minimum stellar mass 0.08 M⊙ and maximum stellar mass 100 M⊙
binary_model = SFH.RandomBinaryPairs(0.4) # Model for including unresolved photometric binaries
mag_lim, mag_lim_name = 32.0, "F606W" # Sampled stars will only be returned if they are brighter than 32 mag in F606W (more efficient)
# Now we sample the stars
stars = SFH.generate_stars_mass_composite(mini_vec, mags, filters, mstar, ssp_massfracs, imf; 
            binary_model=binary_model, dist_mod=dmod, mag_lim=mag_lim, mag_lim_name=mag_lim_name);

Now stars[1] will contain initial mass information of the samples stars and stars[2] will contain photometry.

In [9]:
# Concatenate along first axis (combines results from all SSPs into one vector.
# The first entry in each element is primary star mass, second entry is secondary star mass (if binary)
stars_mini = reduce(vcat, stars[1])

1639528-element Vector{SVector{2, Float64}}:
 [0.7446468026260441, 0.1894120003013573]
 [0.6644104351286498, 0.10961324463901075]
 [0.5713547022937803, 0.0]
 [0.9848200286728362, 0.49716764254058604]
 [0.5556230350980677, 0.1647312487247134]
 [0.5589410954637473, 0.0]
 [1.6733922170741877, 0.39562678199815965]
 [0.6197467522038733, 0.0]
 [0.5135365485558637, 0.0]
 [0.68709978808676, 0.20550308360709882]
 [0.636090591290791, 0.2871786890359084]
 [2.9669610143988416, 0.7544823998424773]
 [0.5945033546269302, 0.42018440547120306]
 ⋮
 [1.1207238617496933, 0.0]
 [0.7468747068161069, 0.0]
 [1.3521425851902817, 0.11440260739023152]
 [0.9889040634072972, 0.09895751923971755]
 [0.9422612367654395, 0.3274658499027239]
 [0.8195788873955737, 0.0]
 [4.6605230519631835, 0.0]
 [0.7114265868095565, 0.1760713660586008]
 [0.6939509341679061, 0.11573606872221336]
 [1.1216613602704135, 0.9012628176917526]
 [0.6349118233518125, 0.1103000066373738]
 [0.8340500114150776, 0.0]

In [10]:
stars_mags = reduce(vcat, stars[2])

1639528-element Vector{SVector{12, Float64}}:
 [29.472093270260654, 29.329105299264427, 29.227561231142545, 29.02871412701405, 29.101626214015152, 28.951648982810077, 28.82596531968176, 28.659194585305308, 28.705573082324058, 28.546125321676833, 28.51402251127052, 28.429747800036353]
 [31.162256607575472, 30.961572148653257, 30.82794386147129, 30.572511223774114, 30.665525138436315, 30.473257549086433, 30.31782099179218, 30.11755505335293, 30.171490223080177, 29.975188349213497, 29.9326876045382, 29.821981251062788]
 [32.234460986526535, 31.96108316201577, 31.79227663162989, 31.47565623013291, 31.59098150506813, 31.352935561788513, 31.167378582047434, 30.93813095470136, 30.998937904770216, 30.760929182709287, 30.707232389978934, 30.567802423661682]
 [33.15051789718469, 32.81899870396036, 32.628465171436126, 32.25242925749775, 32.389377982987256, 32.11017706078899, 31.899212274097522, 31.647793657987428, 31.71372784946184, 31.439501532000776, 31.375752726643956, 31.21071726447984]
 [32.

In [11]:
# We can combine these into a TypedTables.Table,
mass_table = Table(NamedTuple{(:primary_mass, :secondary_mass)}(Tuple(i)) for i in stars_mini)

Table with 2 columns and 1639528 rows:
      primary_mass  secondary_mass
    ┌─────────────────────────────
 1  │ 0.744647      0.189412
 2  │ 0.66441       0.109613
 3  │ 0.571355      0.0
 4  │ 0.98482       0.497168
 5  │ 0.555623      0.164731
 6  │ 0.558941      0.0
 7  │ 1.67339       0.395627
 8  │ 0.619747      0.0
 9  │ 0.513537      0.0
 10 │ 0.6871        0.205503
 11 │ 0.636091      0.287179
 12 │ 2.96696       0.754482
 13 │ 0.594503      0.420184
 14 │ 0.605317      0.521958
 15 │ 0.624257      0.0
 16 │ 1.138         0.562942
 17 │ 0.618191      0.282447
 18 │ 0.707676      0.490091
 19 │ 0.531747      0.285337
 20 │ 0.530323      0.1721
 21 │ 0.712508      0.226736
 22 │ 0.540579      0.215558
 23 │ 0.763758      0.320799
 ⋮  │      ⋮              ⋮

In [12]:
mag_table = Table(NamedTuple{filternames(bcg)}(Tuple(i)) for i in stars_mags)

Table with 12 columns and 1639528 rows:
      F435W    F475W    F502N    F550M    F555W    F606W    F625W    F658N    ⋯
    ┌──────────────────────────────────────────────────────────────────────────
 1  │ 29.4721  29.3291  29.2276  29.0287  29.1016  28.9516  28.826   28.6592  ⋯
 2  │ 31.1623  30.9616  30.8279  30.5725  30.6655  30.4733  30.3178  30.1176  ⋯
 3  │ 32.2345  31.9611  31.7923  31.4757  31.591   31.3529  31.1674  30.9381  ⋯
 4  │ 33.1505  32.819   32.6285  32.2524  32.3894  32.1102  31.8992  31.6478  ⋯
 5  │ 32.3855  32.0989  31.9257  31.5925  31.7136  31.4637  31.2708  31.0348  ⋯
 6  │ 32.36    32.0787  31.9063  31.5828  31.7006  31.4575  31.2687  31.0365  ⋯
 7  │ 34.3493  33.9626  33.775   33.2869  33.4574  33.1125  32.8667  32.5879  ⋯
 8  │ 31.7015  31.4637  31.3113  31.0246  31.129   30.9132  30.742   30.526   ⋯
 9  │ 32.8829  32.5678  32.3822  32.0257  32.1556  31.8893  31.6861  31.4413  ⋯
 10 │ 30.79    30.6086  30.4859  30.2456  30.3331  30.1521  30.0045  29.8122  ⋯


In [13]:
# Concatenate the tables
stars_table = Table(mass_table, mag_table)

Table with 14 columns and 1639528 rows:
      primary_mass  secondary_mass  F435W    F475W    F502N    F550M    ⋯
    ┌────────────────────────────────────────────────────────────────────
 1  │ 0.744647      0.189412        29.4721  29.3291  29.2276  29.0287  ⋯
 2  │ 0.66441       0.109613        31.1623  30.9616  30.8279  30.5725  ⋯
 3  │ 0.571355      0.0             32.2345  31.9611  31.7923  31.4757  ⋯
 4  │ 0.98482       0.497168        33.1505  32.819   32.6285  32.2524  ⋯
 5  │ 0.555623      0.164731        32.3855  32.0989  31.9257  31.5925  ⋯
 6  │ 0.558941      0.0             32.36    32.0787  31.9063  31.5828  ⋯
 7  │ 1.67339       0.395627        34.3493  33.9626  33.775   33.2869  ⋯
 8  │ 0.619747      0.0             31.7015  31.4637  31.3113  31.0246  ⋯
 9  │ 0.513537      0.0             32.8829  32.5678  32.3822  32.0257  ⋯
 10 │ 0.6871        0.205503        30.79    30.6086  30.4859  30.2456  ⋯
 11 │ 0.636091      0.287179        31.4873  31.2579  31.1113  30.8253  

From here you can apply completeness and photometric error models with `StarFormationHistories.model_cmd` if you wish.

In [14]:
# This can now be written to a file, for example with CSV.jl
# formatting options exist (e.g., to reduce precision of numbers in the output)
# import CSV
# CSV.write("output.txt", stars_table; delim=' ')